<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;a:link{color: white}">
    <h1 style='color:GhostWhite;'>Part 2: Should This Loan be Approved or Denied ?</h1>
    This is a continuation of notebook <a style="color:yellow" href="https://www.kaggle.com/code/josephramon/sba-xgboost-model">Part 1: Should This Loan Be Approved or Denied ?</a><br><br>
    The topic covered here is :<br>
    <p style="color:Gold;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>XGBoost v1.6+ HyperParameter Tuning using Optuna - Full and Incremental</b></p>
    </div>

<div class="alert alert-block alert-success">  
    <b>Dataset Source</b><br><br>
    <a href="https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied">U.S. Small Business Administration (SBA) Dataset</a>
<br><br>
    All information about the dataset can be found at the <b>above link</b><br><br>    
    *<i>Thanks to Hamza for his <a href="https://www.kaggle.com/code/hamzaghanmi/xgboost-hyperparameter-tuning-using-optuna/notebook">Notebook on Optuna</a> which was used as a guide.</i> 
<br><br>
    If interested, Data Exploratory Visualization in Tableau can also be seen at :<br>
    <a href= "https://public.tableau.com/app/profile/joseph8038/viz/SBADatasetVisualizationandAnalysis/SBADatasetVisualizationandAnalysis-StoryBoard">SBA Data Exploratory Visualization in Tableau</a>
</div>

<div class="alert alert-block alert-info" style="color:DarkSlateBlue">
This notebook is divided into 2 main parts:<br>
<ul>
<li><a style="color:DarkSlateGrey;" href="#part1"><b>Part 1: XGBoost HyperParameter Tuning using Optuna - Full and Incremental</b></a></li><br>
<li><a style="color:DarkSlateGrey;" href="#part2"><b>Part 2: Miscellaneous</a></b>  - Early Stopping Rounds, Random Forest Classifier</li>
</ul>
<br>
    <p style="color:FireBrick;"><b>* Output from <a style="color:DarkGoldenRod;" href = "https://www.kaggle.com/code/josephramon/sba-xgboost-model">Part 1 notebook</a> are Input to this notebook.</b></p>
</div>

<h2>Table Of Contents</h2>
<ul>
    <li><a style="color:DarkSlateGrey" href="#paths_and_flags">Paths and Flags</a></li>
    <li><a style="color:DarkSlateGrey" href="#libraries">Libraries</a></li>   
    <li><a style="color:DarkSlateGrey" href="#functions">Custom Functions And Classes</a></li>
    <li><a style="color:DarkSlateGrey" href="#metrics">Metrics Function</a></li>
    <li><a style="color:DarkSlateGrey" href="#xgboost_class">XGBoost Class</a></li>
    <li><a style="color:DarkSlateGrey" href="#other_models">Other Models Class</a></li>
    <li><a style="color:DarkSlateGrey" href="#optuna_class">Optuna Class - for both full datasets or incremental</a></li>
    <br>
    <li><a style="color:DarkSlateGrey" href="#part1">Part 1. XGBoost HyperParameter Tuning using Optuna</a></li>
    <ul>
    <li><a style="color:DarkSlateGrey" href="#full_df">Optuna Study : Full Dataset</a></li>
    <li><a style="color:DarkSlateGrey" href="#incremental_df">Optuna Study : Incremental Dataset</a></li>
    <li><a style="color:DarkSlateGrey" href="#optuna_metrics">Optuna Study Metrics</a></li>    
    <li><a style="color:DarkSlateGrey" href="#try_best_hp">Model v4 : Try the Optuna Hyperparameters</a></li>
    <li><a style="color:DarkSlateGrey" href="#optuna_comparison">Optuna Tuning Comparison</a></li>
    <li><a style="color:DarkSlateGrey" href="#cross_validation">Cross Validation</a></li>
    </ul>
    <br>
    <li><a style="color:DarkSlateGrey" href="#part2">Part 2. Miscellaneous</a></li>
    <ul>
    <li><a style="color:DarkSlateGrey" href="#early_stopping_rounds">Early Stopping Rounds</a></li>
    <li><a style="color:DarkSlateGrey" href="#random_forest_classifier">Random Forest Classifier</a></li>
    </ul>
</ul>

<a id="paths_and_flags"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Paths and Flags</b></div>

In [1]:
import os
'''
kaggle_flag :
   0 - if running outside Kaggle (e.g. Jupyter Notebook), change filepath & savepath to your 
       own path
   1 - if running as a Kaggle notebook
'''
# Change this logic to your own if needed
if os.path.exists('../usr/lib/myfuncs/myfuncs.py'):
    kaggle_flag = 1
    print('Running a Kaggle notebook')
else:
    kaggle_flag = 0
    print('Not running a Kaggle notebook')

# alert_flag - change to 0 for no sound alert, 1 for sound alert after long running cells
alert_flag = 0

'''
We have two options for running Optuna tuning on XGBoost:  
   OptunaStudy() - run Optuna on the full dataset
   OptunaStudyChunk() - run in chunks, lighter on memory, but much slower

Change flag below as needed:
   1 to run OptunaStudy() only
   2 to run OptunaStudyChunk() only
   3 to run both
'''
optuna_flag = 1

# GPU is automatically detected if activated

#---------------------------------------------------------------------------------------#

if kaggle_flag == 1:             # Kaggle
    filepath  = "../input/sba-xgboost-model/"
    savepath  = "./"
    final_ds  = '../input/sba-xgboost-model/sba_final.csv.feather'  # imported from Part 1 Notebook
    final_csv = '../input/sba-xgboost-model/sba_final.csv'          # imported from Part 1 Notebook
    functions_path = "../usr/lib/myfuncs/myfuncs.py"
else:
    filepath  = "C:\\Python\\Python_Data_Science_Exercises\\datasets\\"
    savepath  = "C:\\Python\\Python_Data_Science_Exercises\\datasets\\"
    final_ds  = f'{savepath}sba_final.csv.feather'
    final_csv = f'{savepath}sba_final.csv'
    functions_path = 'C:\\Python\\Python_Data_Science_Exercises\\mylibs\\'

audio_path="https://www.soundjay.com/misc/sounds/tablet-bottle-1.mp3" # for alert

<a id="libraries"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Libraries</b></div>

In [29]:
from IPython.display import clear_output   # to be able to use clear_output(wait=True)
def install_packages():
    print('Please wait, package installations started, if needed')
    libs = ['scikit-learn', 'seaborn', 'numpy','matplotlib', 'tensorflow','torch','joblib',
            'psutil','imbalanced-learn','xgboost','optuna','pyarrow','pyttsx3',
            'pympler','memory_profiler','line_profiler','sweetviz']
    
    piplist = !pip list
    for i in range(len(libs)):
        if not piplist.grep(libs[i]):
            !pip3 install {libs[i]}

    clear_output(wait=True)
    print('Package installations completed')

install_packages()

Package installations completed


In [30]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import pyttsx3
from IPython.display import Audio, display
from IPython.display import FileLink
from IPython.display import IFrame
from IPython.core.display import HTML
import hashlib
import copy                     # for deepcopy()
import datetime as dt
import optuna
import gc
import shutil
import psutil
import os
import sys
import pickle
import joblib
from pprint import pprint
import torch                    # for clearing GPU cache
from time import sleep
import multiprocessing as mp
from pympler import muppy       # for memory profiling
from pympler import summary     # for memory profiling
from pympler.classtracker import ClassTracker
import xgboost
%load_ext line_profiler
%load_ext memory_profiler
%matplotlib inline
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

clear_output(wait=True)
print('Package imports completed')

ModuleNotFoundError: No module named 'cuml'

In [3]:
# Kernel must be restarted if XGBoost is upgraded
# importlib.reload and %autoreload do not work, so manually restart 
# This check is basically for Kaggle which has an older version of XGBoost, at least at this time
if '1.6' not in xgboost.__version__:
    !pip3 install --upgrade xgboost
    clear_output(wait=True)
    print('XGBoost Package upgrade completed.  KERNEL RESTART NEEDED FOR NOTEBOOK.')
else:
    print('XGBoost version is already 1.6+')


XGBoost Package upgrade completed


In [ ]:
# XGBoost version should be 1.6+ and up
assert '1.6' in xgboost.__version__,\
    "XGBoost version must be 1.6+. RESTART KERNEL if already upgraded."

print(f'XGBoost __Version__ : {xgboost.__version__}')
print()
!pip3 show xgboost

In [6]:
# ensure garbage collector is enabled
(gc.isenabled() == False) and gc.enable();

<a id="functions"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Custom Functions and Classes</b></div>

In [ ]:
sys.path

In [7]:
# import custom functions
# RESTART kernel if myfuncs is modified
if functions_path not in sys.path:
    sys.path.append(functions_path)
from myfuncs import *

print('Custom functions import completed')

Custom functions import completed


<div class="alert alert-block alert-info">
<b>Custom functions and classes in <a style="color:ForestGreen" href="https://www.kaggle.com/code/josephramon/myfuncs" target="_blank">myfuncs.py</a></b>.<br>  
In Kaggle, myfuncs.py is set up as a <b>Utility Script</b> in /usr/lib<br>
<ul>
    <li>is_kaggle_gpu_enabled()</li>
<li>clear_gpu(tree_method='gpu_hist')</li>
<li>reduce_mem_usage(df, print_info = True, use_float16=False)</li>
<li>runtime(rt1,rt2)</li>
<li>create_download_link(title = "Download ", filename = "data.csv")</li>
<li>GetRam()</li>
<li>convertFloatToDecimal(f=0.0, precision=2)</li>
<li>formatFileSize(size, sizeIn, sizeOut, precision=0)</li>
<li>check_cols_with_nulls(df)</li>
<li>check_infinity_nan(df, dfname)</li>
<li>fixvals(val)</li>
<li>model_eval(y_valid,predictions, cmDisplay='False')</li>
<li>plot_features(booster, figsize)</li>
<li>make_mi_scores(X, y)</li>
<li>plot_mi_scores(scores)</li>
<li>GetSweetVizReport(df, savepath, kaggle_flag)</li>
<li>class color
</div>

In [8]:
gpu_enabled = is_kaggle_gpu_enabled()

if gpu_enabled == False:
    tree_method = 'hist'
else:
    tree_method = 'gpu_hist'

del gpu_enabled
gc.collect()

sleep(5)
clear_output(wait=True)
tree_method

'hist'

In [12]:
''' 
Set up voice object.  Used in different areas of notebook to indicate completion of long processes.
'''
if kaggle_flag == 0:   # not Kaggle
    engine = pyttsx3.init()  # object creation

    """ RATE"""
    #rate = engine.getProperty('rate')   # getting details of current speaking rate
    #print (rate)                        #printing current voice rate
    engine.setProperty('rate', 175)     # setting up new voice rate

    """VOLUME"""
    #volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
    #print (volume)                         #printing current volume level
    engine.setProperty('volume',0.7)        # setting up volume level  between 0 and 1

    """VOICE"""
    voices = engine.getProperty('voices')       #getting details of current voice
    #engine.setProperty('voice', voices[0].id)  #changing index, changes voices. o for male
    engine.setProperty('voice', voices[1].id)   #changing index, changes voices. 1 for female

<a id="xgboost_class"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>XGBoost Class</b></div>

In [13]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split
#from xgboost import XGBRegressor
from xgboost import XGBClassifier

class process_model():  
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.X_train, self.y_train = None, None
        self.X_valid, self.X_test = None, None
        self.y_valid, self.y_test = None, None

        print(f'MIS_Status Count ->  1 : {Counter(y)[1]}, 0 : {Counter(y)[0]}')
    
    # oversampling method
    def osample(self, print_info = True):
        # define oversampling strategy
        oversample = RandomOverSampler(sampling_strategy='minority') 
        if print_info == True:
            print('X size : ', len(self.X))
            print('y size : ', len(self.y))
        # fit and apply the transform
        X_over, y_over = oversample.fit_resample(self.X, self.y)

        # summarize class distribution
        if print_info == True:
            print(f'Before Oversampling -> 1 : {Counter(self.y)[1]}, 0 : {Counter(self.y)[0]}')
            print(f'After Oversampling  -> 1 : {Counter(y_over)[1]}, 0 : {Counter(y_over)[0]}')
        
        # update X and y with the oversampled results 
        self.X = X_over
        self.y = y_over
        
        # return the oversampled results in case they are needed in another module
        #return {'X_over':X_over, 'y_over':y_over}
    
    def split_data(self, X_size = 0.7):   
        # Split Data into Train:Validate:Test
        
        # train_size=X_size
        # In the first step, we will split the data in training and remaining dataset
        self.X_train, X_rem, self.y_train, y_rem = train_test_split(self.X, self.y,
                                                        train_size = X_size, random_state=48) 

        # Now since we want the valid and test size to be equal,
        # we have to define valid_size=0.5 (that is 50% of remaining data)
        # test_size = 0.5

        self.X_valid, self.X_test, self.y_valid, self.y_test = train_test_split(X_rem,y_rem,
                                                            test_size=0.5, random_state=48)
        
        return {'X_train':self.X_train, 'y_train':self.y_train,
                'X_valid':self.X_valid, 'y_valid':self.y_valid,
                'X_test':self.X_test, 'y_test':self.y_test}
    
    # Method to run model 
    # desc - description of metrics report
    def prep_run_model(self, desc='Metrics', cmDisplay=False, PipeLine_flag = False,
                hyperparams = {'n_estimators': 1000, 'learning_rate': 0.05, 'max_depth': 6,
                               'tree_method':tree_method, 'early_stopping_rounds':100,
                               'eval_metric':['logloss','error']}):
        # from XGBoost 1.6, early_stopping_rounds and eval_metric are under parameters,
        # and deprecated from fit() method.
        # The default hyperparameters are conservative, to help avoid overfitting
        
        print()
        print(f"{color.bold}Please wait, Fitting model can take time ...{color.end}")
        
        '''
        XGBRegressor is for continuous target/outcome variables. These are often called 
        "regression problems."

        XGBClassifier is for categorical target/outcome variables. These are often called 
        "classification problems."
        
        xg_model = XGBRegressor(n_estimators = self.mn_estimators,
                                learning_rate = self.mlearning_rate,
                                max_depth = self.mmax_depth,
                                n_jobs=4)
        
        xg_model = XGBClassifier(n_estimators = self.mn_estimators,
                                learning_rate = self.mlearning_rate,
                                max_depth = self.mmax_depth,
                                use_label_encoder =False,
                                n_jobs=4)
        '''
        
        if PipeLine_flag == True:
            # hyperparams is a result of Optuna hyperparameter tuning (Part 3 of this notebook)
            # the hyperparameters lean towards being conservative to help avoid overfitting
            hyperparams = { 'tree_method': 'gpu_hist',
                            'lambda': 0.023437933789759252,
                            'alpha': 0.005813454622750776,
                            'gamma': 0,
                            'colsample_bytree': 0.9,
                            'subsample': 1.0,
                            'learning_rate': 0.05,
                            'n_estimators': 1000,
                            'max_depth': 13,
                            'random_state': 48,
                            'min_child_weight': 1,
                            'early_stopping_rounds': 100.0,
                            'eval_metric': 'error'
                          }
            
        xg_model = XGBClassifier(**hyperparams,use_label_encoder =False)
       
        #eval_setparam = [(self.X_train, self.y_train), (self.X_valid, self.y_valid)]
        eval_setparam = [(self.X_valid, self.y_valid)]
        
        xg_model.fit(self.X_train, self.y_train, 
                     eval_set = eval_setparam,
                     verbose=False)
        
        gc.collect()
        clear_gpu()
 
        print("Fitting model completed.")
        print()
        print('Preparing Predictions')
    
        # Get predictions
        predictions = xg_model.predict(self.X_valid)
    
        print()
        print(f'{color.underline}{desc}{color.end}')

        eval_results = model_eval(self.y_valid, predictions, cmDisplay)
            
        # Return these values as they will be needed for further testing or metrics
        # in dictionary form to remember easier 
        return {'xg_model':xg_model,'predictions':predictions,
                    'X_train':self.X_train, 'y_train':self.y_train,
                    'X_valid':self.X_valid, 'y_valid':self.y_valid,
                    'X_test':self.X_test, 'y_test':self.y_test, 'eval_results':eval_results}

<a id="other_models"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 5px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Other Models Class</b></div>

In [42]:
from sklearn.ensemble import RandomForestClassifier
if tree_method == 'gpu_hist':
    import cuml
    from cuml.ensemble import RandomForestClassifier as cuRF

# inherit from XGBoost class (process_model)
class other_models(process_model):  
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.X_train, self.y_train = None, None
        self.X_valid, self.X_test = None, None
        self.y_valid, self.y_test = None, None

    #    print(f'MIS_Status Count ->  1 : {Counter(y)[1]}, 0 : {Counter(y)[0]}')
    
    # Method to run model 
    # desc - description of metrics report
    def prep_run_model(self, desc='Metrics', modelname = 'rfc',
                       hparams = {'n_estimators':100, 'random_state':48, 'max_depth':10},
                       cmDisplay=False):
        print()
        print(f"{color.bold}Please wait, Fitting model can take time ...{color.end}")  

        if modelname == 'rfc':
            if tree_method == 'gpu_hist':
                hparams.update({'max_features':1.0,
                                'n_bins':8})
                model = cuRF(**hparams)
            else:
                hparams.update({'max_features':1.0,
                                'n_bins':8})
                model = RandomForestClassifier(**hparams) 
            model.fit(self.X_train, self.y_train)
            
        print("Fitting model completed.")
        print()
        print('Preparing Predictions')
    
        # Get predictions
        predictions = model.predict(self.X_valid)
        print(f'{color.underline}{desc}{color.end}')
    
        if modelname == 'rfc':
            if tree_method == 'gpu_hist':
                print()
                cu_score = cuml.metrics.accuracy_score( self.y_valid, predictions )
                print(f'{color.bdgreen}cuml Score : {round(cu_score * 100,2)} %{color.end}')
                print()
            
        eval_results = model_eval(self.y_valid, predictions, cmDisplay)
        print()
        print
        
        gc.collect()
        clear_gpu()
        
        # Return these values as they will be needed for further testing or metrics
        # in dictionary form to remember easier 
        return {'model':model,'predictions':predictions,
                    'X_train':self.X_train, 'y_train':self.y_train,
                    'X_valid':self.X_valid, 'y_valid':self.y_valid,
                    'X_test':self.X_test, 'y_test':self.y_test, 'eval_results':eval_results}

<a id="optuna_class"></a>
<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Optuna Class</b><br>
This is for both full dataset or incremental dataset trials.  There are two objective functions here,objective() and objective_batch()</div>

In [43]:
class optuna_tuning(process_model):  
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.X_train, self.y_train = None, None
        self.X_valid, self.X_test = None, None
        self.y_valid, self.y_test = None, None
    
    # for printing only the best values, saves memory too
    def logging_callback(self, study, frozen_trial):
        previous_best_value = study.user_attrs.get("previous_best_value", None)
        if previous_best_value != study.best_value:
            study.set_user_attr("previous_best_value", study.best_value)
            clear_output(wait=True)
            self.text_out="{}Trial {} done with best value: {}{}{} and parameters: {}{}. ".format(
                color.bdblue,
                frozen_trial.number,
                color.bdgreen,
                frozen_trial.value,
                color.bdblue,
                color.end,
                frozen_trial.params
                )
            print(self.text_out)
            
            # Writing to file
            with open(f"{savepath}xgb_optuna_study_log.txt", "a") as os_log:
                os_log.write('\n\n')
                os_log.write(f"Current Ram Used: {GetRam()} %\n")
                os_log.write(self.text_out)
    
    # save study
    def save_study(self, study, frozen_trial):
        joblib.dump(study, f"{savepath}xgb_optuna_study_callbacks.pkl")   # save study
        #print(f'{color.bdblue}Current study saved for Trial {frozen_trial.number}{color.end}')
    
    # for tuning full dataset
    def objective(self, trial, gt, n_estimators = 1000):        
        nn_early_stopping_rounds = n_estimators * 0.1
        # the hyperparameters lean towards being conservative to help avoid overfitting
        param = {
            # tree_method would ideally be gpu_hist for faster speed
            'tree_method':trial.suggest_categorical('tree_method', [tree_method]), 
            # L2 regularization weight, Increasing this value will make model more conservative
            'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
            # L1 regularization weight, Increasing this value will make model more conservative
            'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
            # Min loss reduction for further partition on a leaf node. larger,the more conservative
            'gamma':trial.suggest_categorical('gamma', [0,3,6]),
            # sampling according to each tree
            'colsample_bytree': trial.suggest_categorical('colsample_bytree',
                            [0.6,0.7,0.8,0.9,1.0]),
            #                [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
            # sampling ratio for training data
            #'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,0.9,1.0]),
            'learning_rate': trial.suggest_categorical('learning_rate',
                            [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02,0.05]),
            'n_estimators': trial.suggest_categorical('n_estimators',[n_estimators]),
            # maximum depth of the tree, signifies complexity of the tree
            'max_depth': trial.suggest_categorical('max_depth', [9,11,13]),
            'random_state': trial.suggest_categorical('random_state', [48]),
            # minimum child weight, larger the term more conservative the tree
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'early_stopping_rounds': trial.suggest_categorical('early_stopping_rounds',
                                          [nn_early_stopping_rounds]),                     
            'eval_metric': trial.suggest_categorical('eval_metric', [['logloss','error']])
        }
        if GetRam() >= 90:
            raise MemoryError('Short On Memory')
            
        # clear outputs, reprint saved data
        if trial.number > 0:
            clear_output(wait=True)
            gc.collect()
            print(self.text_out)
            
        # print(param)  # for debugging, comment out if desired
        model_xgbc = XGBClassifier(**param, use_label_encoder =False)  
    
        print()
        print(f"Current Ram Used: {GetRam()} %")
        rt2=dt.datetime.now()
        print(f'{color.bdgreen}Total Elapsed Time from Training Start: {color.end}', end='')
        runtime(gt, rt2)  
        print(f'Running Trial {trial.number}')
            
        model_xgbc.fit(self.X_train, self.y_train, eval_set=[(self.X_valid, self.y_valid)],
                    verbose=False)

        preds = model_xgbc.predict(self.X_valid)
    
        rmse = metrics.mean_squared_error(self.y_valid, preds,squared=False)
 
        trial.report(rmse, 1)
            
        if trial.should_prune():
            text_prune = f'{color.bold}Trial {trial.number} pruned{color.end}'
            # Writing to file
            with open(f"{savepath}xgb_optuna_study_log.txt", "a") as os_log:
                os_log.write('\n')
                os_log.write(text_prune)
            del model_xgbc, preds, text_prune
            gc.collect()
            sleep(3)
            clear_gpu()
            raise optuna.TrialPruned()

        text_dtl = "Trial {} finished with parameters: {}. ".format(
            trial.number,
            trial.params
            )
        # Writing to file
        with open(f"{savepath}xgb_optuna_study_log.txt", "a") as os_log:
            os_log.write('\n\n')
            os_log.write(f"Current Ram Used: {GetRam()} %\n")
            os_log.write(text_dtl)
            
        rt2=dt.datetime.now()

        del model_xgbc, preds, text_dtl
        gc.collect()
        sleep(3)       
        clear_gpu()        

        return rmse
 

    # for tuning incrementally in chunks
    def objective_chunk(self, trial, n_trials, gt,
                        n_chunksize = 200000, n_estimators = 1000):
        nn_early_stopping_rounds = n_estimators * 0.1
        
        # the hyperparameters lean towards being conservative to help avoid overfitting
        param = {
            # tree_method would ideally be gpu_hist for faster speed
            'tree_method':trial.suggest_categorical('tree_method', [tree_method]), 
            # L2 regularization weight, Increasing this value will make model more conservative
            'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
            # L1 regularization weight, Increasing this value will make model more conservative
            'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
            # Min loss reduction for further partition on a leaf node. larger,the more conservative
            'gamma':trial.suggest_categorical('gamma', [0,3,6]),
            # sampling according to each tree
            'colsample_bytree': trial.suggest_categorical('colsample_bytree',
                                [0.6,0.7,0.8,0.9,1.0]),
            #                [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
            # sampling ratio for training data
            #'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,0.9,1.0]),
            'learning_rate': trial.suggest_categorical('learning_rate',
                            [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02,0.05]),
            'n_estimators': trial.suggest_categorical('n_estimators',[n_estimators]),
            # maximum depth of the tree, signifies complexity of the tree
            'max_depth': trial.suggest_categorical('max_depth', [9,11,13]),
            'random_state': trial.suggest_categorical('random_state', [48]),
            # minimum child weight, larger the term more conservative the tree
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'early_stopping_rounds': trial.suggest_categorical('early_stopping_rounds',
                                          [nn_early_stopping_rounds]),                     
            'eval_metric': trial.suggest_categorical('eval_metric', [['logloss','error']])
        }
        if GetRam() >= 90:
            raise MemoryError('Short On Memory')
        
        model_xgbc = XGBClassifier(**param,use_label_encoder =False)  
    
        rt1=dt.datetime.now()
               
        # clear outputs, reprint saved data
        if trial.number > 0:
            clear_output(wait=True)
            gc.collect()
            print(self.text_out)
            
            if os.path.exists(f'{savepath}model_xgbc.json'):
                file_size = os.path.getsize(f'{savepath}model_xgbc.json')
                file_size = formatFileSize(file_size,'B','MB',2)
                print(f"Current Model File Size : {file_size}MB")

        '''
        For batch, use xgb_model parameter in fit().  There are two ways :
           1. save the model to a file, after 1st trial, then give the name to the next trials
           2. just give the name of the model object, in this case model_xgbc
        '''
    
        #X_valid_list, y_valid_list = [],[]
        # Fit Model
        for i, self.X in enumerate(pd.read_csv(final_csv, chunksize = n_chunksize), start = 1):
            self.X = reduce_mem_usage(self.X, print_info=False)
            self.y = self.X.pop('MIS_Status')

            self.osample(print_info = False)
            self.split_data(0.7)
            
            #X_valid_list.append(self.X_valid.copy())
            #y_valid_list.append(self.y_valid.copy())
            
            #X_valid = pd.concat(X_valid_list)
            #y_valid = pd.concat(y_valid_list)
            X_valid = self.X_valid
            y_valid = self.y_valid
        
            self.X, self.y = None, None
            self.X_valid, self.y_valid = None, None
            self.X_test, self.y_test = None, None

            if i == 1:            
                print()
                print(f"Current Ram Used: {GetRam()} %")
                rt2=dt.datetime.now()
                print(f'{color.bdgreen}Total Elapsed Time from Training Start: {color.end}', end='')
                runtime(gt, rt2)  
                print(f'Running Trial {trial.number} Chunk: {i}',end = ' | ')
                model_xgbc.fit(self.X_train, self.y_train, eval_set=[(X_valid, y_valid)],
                        verbose=False)
            else:
                print(f'{i}',end = ' | ')
                model_xgbc = XGBClassifier(use_label_encoder =False)
                model_xgbc.load_model(f'{savepath}model_xgbc.json')
                
                #if i == 2:
                #    param.pop('tree_method')
                #    param.update({'updater':'refresh',
                #                'process_type': 'update',
                #                'refresh_leaf': True})
                #    model_xgbc.set_params(**param)
                
                model_xgbc.fit(self.X_train, self.y_train, eval_set=[(X_valid, y_valid)],
                        verbose=False, xgb_model = model_xgbc
                        # uncomment below if you want to use a saved file
                        #xgb_model = f'{savepath}model_xgbc.json'
                        )

            '''Auxiliary attributes of the Python Booster object (such as feature_names) will 
            not be saved when using binary format. To save those attributes, use JSON instead.'''
            # uncomment below if using a saved file
            #model_xgbc.get_booster().save_model(f'{savepath}model_xgbc.json')
            model_xgbc.save_model(f'{savepath}model_xgbc.json')
        
            #X_valid = pd.concat(X_valid_list)
            #y_valid = pd.concat(y_valid_list)
        
            preds = model_xgbc.predict(X_valid)
    
            rmse = metrics.mean_squared_error(y_valid, preds,squared=False)
        
            del model_xgbc
            self.X_train, self.y_train = None, None
            gc.collect()
            sleep(5)
            clear_gpu()
            
            trial.report(rmse, i)
            
            if trial.should_prune():
                # Writing to file
                with open(f"{savepath}xgb_optuna_study_log.txt", "a") as os_log:
                    os_log.write('\n')
                    os_log.write(f'{color.bold}Trial {trial.number} pruned{color.end}')
                    os_log.write('\n')
                raise optuna.TrialPruned()
        
        if os.path.exists(f'{savepath}model_xgbc.json'):
            file_size = os.path.getsize(f'{savepath}model_xgbc.json')
            file_size = formatFileSize(file_size,'B','MB',2)
            print(f"Current Model File Size : {file_size}MB")
                
        # Writing to file
        with open(f"{savepath}xgb_optuna_study_log.txt", "a") as os_log:
            os_log.write('\n\n')
            os_log.write(f"Current Ram Used: {GetRam()} %")
            if os.path.exists(f'{savepath}model_xgbc.json'):
                os_log.write(f" | Current Model File Size : {file_size}MB\n")
            os_log.write(f"Trial {trial.number} finished with parameters: {trial.params}.")
            
        rt2=dt.datetime.now()

        self.X_train, self.y_train = None, None
        gc.collect()
        sleep(5)
        clear_gpu()
            
        return rmse

    # Define an objective function for Random Forest
    def objective_rf(self, trial, gt):
        hparams = {
            'max_features': trial.suggest_uniform('max_features', 0.15, 1.0),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 14),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 14),
            'max_samples': trial.suggest_uniform('max_samples', 0.6, 0.99),
            'max_depth': trial.suggest_categorical('max_depth', [5,10,15]),
            'n_estimators': trial.suggest_categorical('n_estimators', [100,250,500])
            # warm_start = True   # for incremental learning  
        }
 
        # clear outputs, reprint saved data
        if trial.number > 0:
            clear_output(wait=True)
            gc.collect()
            print(self.text_out)
            
        if GetRam() >= 90:
            raise MemoryError('Short On Memory')
            
        if tree_method == "gpu_hist":
            model_rf = cuRF(**hparams)
        else:
            model_rf = RandomForestClassifier(**hparams)
        print()
        print(f"Current Ram Used: {GetRam()} %")
        rt2=dt.datetime.now()
        print(f'{color.bdgreen}Total Elapsed Time from Training Start: {color.end}', end='')
        runtime(gt, rt2)  
        print(f'Running Trial {trial.number}')

        ## Fit Model
        model_rf.fit(self.X_train, self.y_train)

        # Report intermediate objective value
        intermediate_value = model_rf.score(self.X_valid, self.y_valid)
        trial.report(intermediate_value, 0)
            
        if trial.should_prune():
            text_prune = f'{color.bold}Trial {trial.number} pruned{color.end}'
            # Writing to file
            with open(f"{savepath}rfc_optuna_study_log.txt", "a") as os_log:
                os_log.write('\n')
                os_log.write(text_prune)
            del model_rf, text_prune
            gc.collect()
            sleep(3)
            clear_gpu()
            raise optuna.TrialPruned()

        text_dtl = "Trial {} finished with parameters: {}. ".format(
            trial.number,
            trial.params
            )
        # Writing to file
        with open(f"{savepath}rfc_optuna_study_log.txt", "a") as os_log:
            os_log.write('\n\n')
            os_log.write(f"Current Ram Used: {GetRam()} %\n")
            os_log.write(text_dtl)
            
        rt2=dt.datetime.now()

        del model_rf, text_dtl
        gc.collect()
        sleep(3)       
        clear_gpu()        

        return intermediate_value

<a id="part1"></a>
<div style="font-family: Trebuchet MS;background-color:DarkRed;color:AliceBlue;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
<h1 style='color:GhostWhite;'>Part 1. XGBoost HyperParameter Tuning using Optuna</h1>
</div>

<a id="full_df"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>1.1 Optuna Study - Full Dataset</h2>
</div>

In [25]:
%%time
        
# For running Optuna tuning on full dataset.
def OptunaStudy():
    X = pd.read_feather(final_ds)
    y = X.pop('MIS_Status')
    
    # instantiate the optuna_tuning class
    ot = optuna_tuning(X, y)
    ot.osample()
    ot.split_data(0.7)
    
    ot.X, ot.y = None, None
    del X, y
    gc.collect()
    sleep(3)
    
    ''' 
    - Initialize the number of trials and n_estimators.
    - Rather than run at a very high n_trials like 100 and above, I prefer to run this function, 
         with an n_trials like 50, three times to get the best_trial values 
    - Recommended n_estimators is 100 for really huge datasets, 1000 for medium-sized.
    - For our dataset here, 1000 is fine.  Ramping it up to 4000 for example will have no
         significant benefits, will be very slow, and a Kaggle notebook will run out of memory often.
    '''
    
    nn_trials = 50
    nn_estimators = 1000
    
    if os.path.exists(f'{savepath}xgb_optuna_study_log.txt'):
        os.remove(f'{savepath}xgb_optuna_study_log.txt')

    # STUDY
    
    # Turn off optuna log notes, to use own logging notes
    optuna.logging.set_verbosity(optuna.logging.WARN)

    print(f"{color.bold}Please wait, finding best trial ...{color.end}")
            
    gt = dt.datetime.now()
        
    study = optuna.create_study(direction='minimize')
    try:
        # callbacks [self.save_study] is to save study in case memory fails
        study.optimize(lambda trial: ot.objective(trial, gt, n_estimators = nn_estimators), 
                        n_trials = nn_trials,
                        callbacks = [ot.logging_callback, ot.save_study],
                        gc_after_trial = True,
                        catch = (RuntimeWarning,ArithmeticError,))
    except MemoryError as e:
        print(f'{color.bdblue}{e} : Memory was getting low, Trial ended early{color.end}')
    
    print()
    print(f'Number of finished trials: {len(study.trials)}')
    print(f'Number of pruned trials: {len(pruned_trials)}')
    print(f'Number of completed trials: {color.bdgreen}{len(complete_trials)}{color.end}')
    print(f'{color.bdblue}Best trial: {study.best_trial.params}{color.end}')
    
    joblib.dump(study, f"{savepath}xgb_optuna_study.pkl")   # save study
    # jl = joblib.load(f"{savepath}xgb_optuna_study.pkl")   # load study

    print()
    return study

if optuna_flag == 1 or optuna_flag == 3:
    study_results = OptunaStudy()

Trial 1 done with best value: 0.2221388462822118 and parameters: {'tree_method': 'hist', 'lambda': 0.004590141566739722, 'alpha': 0.007317728322388094, 'gamma': 0, 'colsample_bytree': 0.6, 'subsample': 1.0, 'learning_rate': 0.014, 'n_estimators': 1000, 'max_depth': 13, 'random_state': 48, 'min_child_weight': 4, 'early_stopping_rounds': 100.0, 'eval_metric': ['error', 'logloss']}. 

Current Ram Used: 65.3 %
Total Elapsed Time from Training Start: Runtime : 0:34:46.310128
Running Trial 16


KeyboardInterrupt: 

<a id="incremental_df"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>1.2 Optuna Study - Incremental Dataset</h2>
</div>

In [44]:
%%time

# For running Optuna tuning incrementally in batches, much slower, but lighter on memory
def OptunaStudyChunk(): 
    ''' 
    - Initialize the number of trials and n_estimators.
    - Rather than run at a very high n_trials like 100 and above, I prefer to run this function, 
         with an n_trials like 50 or 70, three times to get the best_trial values 
    - Recommended n_estimators is 100 for really huge datasets, 1000 for medium-sized.
    - For our dataset here, 1000 is fine.  Ramping it up to 4000 for example will have no
         significant benefits, will be very slow, and a Kaggle notebook will run out of memory often.
    '''

    nn_trials = 3               # n_trials
    nn_chunksize = 200000
    nn_estimators = 1000
    
    if os.path.exists(f'{savepath}xgb_optuna_study_log.txt'):
        os.remove(f'{savepath}xgb_optuna_study_log.txt')

    # get full X_valid, y_valid 
    '''
    print(f"{color.bold}Please wait, getting Validation Data{color.end}")  
    X_valid_list, y_valid_list = [],[]
    for i, X in enumerate(pd.read_csv(final_csv, chunksize = nn_chunksize), start = 1):
        X = reduce_mem_usage(X, print_info=False)
        y = X.pop('MIS_Status')

        oso = optuna_tuning(X,y) 
        oso.osample(print_info = False)
        oso.split_data(0.7)
            
        X_valid_list.append(oso.X_valid.copy())
        y_valid_list.append(oso.y_valid.copy())
            
    X_valid = pd.concat(X_valid_list)
    y_valid = pd.concat(y_valid_list)
        
    del X_valid_list, y_valid_list, oso
    gc.collect() 
    sleep(3)
    '''
        
    # OPTUNA STUDY
    
    # Turn off optuna log notes, to use own logging notes
    optuna.logging.set_verbosity(optuna.logging.WARN)
    
    clear_output(wait=True)
    print(f"{color.bold}Please wait, finding best trial ...{color.end}")
            
    gt = dt.datetime.now()

    otb = optuna_tuning(None, None)
    study = optuna.create_study(direction='minimize')
    try:
        # callbacks [self.save_study] is to save study in case memory fails
        '''
        study.optimize(lambda trial: otb.objective_chunk(trial, nn_trials, gt, X_valid, y_valid,
                                  n_chunksize = nn_chunksize, n_estimators = nn_estimators), 
                        n_trials = nn_trials,
                        callbacks = [otb.logging_callback, otb.save_study],
                        gc_after_trial = True,
                        catch = (RuntimeWarning,ArithmeticError,))
        '''
        study.optimize(lambda trial: otb.objective_chunk(trial, nn_trials, gt,
                                  n_chunksize = nn_chunksize, n_estimators = nn_estimators), 
                        n_trials = nn_trials,
                        callbacks = [otb.logging_callback, otb.save_study],
                        gc_after_trial = True,
                        catch = (RuntimeWarning,ArithmeticError,))
    except MemoryError as e:
        print(f'\n\n{color.bdblue}{e} : Memory was getting low, Trial ended early{color.end}\n\n')
            
    print()
    print(f'Number of finished trials: {len(study.trials)}')
    print(f'Number of pruned trials: {len(pruned_trials)}')
    print(f'Number of completed trials: {color.bdgreen}{len(complete_trials)}{color.end}')
    print(f'{color.bdblue}Best trial: {study.best_trial.params}{color.end}')
    
    joblib.dump(study, f"{savepath}xgb_optuna_study.pkl")   # save study
    # jl = joblib.load(f"{savepath}xgb_optuna_study.pkl")   # load study

    print()
    return study

if optuna_flag == 2 or optuna_flag == 3:
    study_results = OptunaStudyChunk()

CPU times: total: 0 ns
Wall time: 0 ns


<a id="optuna_metrics"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>1.3 Optuna Study Metrics</h2>
</div>

In [ ]:
# uncomment below if you want to see the trials log 
with open(f"{savepath}xgb_optuna_study_log.txt", "r+") as os_log:
    print(os_log.read())

In [ ]:
# uncomment below if you want to load saved study to check, if desired
jl = joblib.load(f"{savepath}xgb_optuna_study.pkl")
jl = jl.best_trial.params
pprint(jl)
# jl.update({'n_estimators': 4000, 'tree_method':tree_method})
'''
best_trial_sorted = {}
for i in sorted (jl) :
    best_trial_sorted.update({i:jl[i]})                          

pprint(best_trial_sorted)
'''

In [ ]:
study_results = joblib.load(f"{savepath}xgb_optuna_study.pkl")
best_trial = study_results.best_trial.params
best_trial['tree_method'] = tree_method
pprint(best_trial)

In [18]:
best_trial = study_results.best_trial.params
#best_trial.update({'n_estimators': 4000, 'tree_method':tree_method})
best_trial['tree_method'] = tree_method
best_trial

{'tree_method': 'hist',
 'lambda': 0.1310350301056629,
 'alpha': 0.13596726922222288,
 'gamma': 0,
 'colsample_bytree': 0.9,
 'subsample': 0.9,
 'learning_rate': 0.05,
 'n_estimators': 1000,
 'max_depth': 13,
 'random_state': 48,
 'min_child_weight': 1,
 'early_stopping_rounds': 100.0,
 'eval_metric': 'logloss'}

In [ ]:
# Trial results dataframe sorted from best value (RMSE) ascending
def ViewResultsAsDf():
    stdf = study_results.trials_dataframe()
    stdf = stdf.sort_values('value',ascending=True)

    return stdf.head(2)    # return here is only used for printing output

ViewResultsAsDf()

In [ ]:
#Visualize parameter importance
optuna.visualization.plot_param_importances(study_results)

In [ ]:
if alert_flag == 1:
    if kaggle_flag == 0:   # not Kaggle
        engine.say("Optuna run completed.")
        engine.runAndWait()
    else:
        display(Audio(url=audio_path, autoplay=True))

<a id="try_best_hp"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>1.4 Model v4 : Try the Optuna Hyperparameters</h2>
</div>

In [19]:
%%time

def RunModelv4():
    X = pd.read_feather(final_ds)
    y = X.pop('MIS_Status')

    model4 = process_model(X, y)
    model4.osample()
    model4.split_data(0.7)
    
    model4.X, model4.y = None, None
    del X, y
    gc.collect()
    sleep(3)
    
    model4_results = model4.prep_run_model("Metrics : After Optuna Tuning",
                                           hyperparams = best_trial)
    
    text_boosted = \
        f"Total boosted rounds: {model4_results['xg_model'].get_booster().num_boosted_rounds()}"
    print(text_boosted)
    
    # save to files for reuse later
    model4_results['xg_model'].save_model(f'{savepath}modelv4.json')
    joblib.dump(model4_results, f"{savepath}model4_results.dict")   

#with warnings.catch_warnings():
#    warnings.filterwarnings('ignore')
#    warnings.simplefilter("ignore")
RunModelv4()

MIS_Status Count ->  1 : 714212, 0 : 154451
X size :  868663
y size :  868663
Before Oversampling -> 1 : 714212, 0 : 154451
After Oversampling  -> 1 : 714212, 0 : 714212

Please wait, Fitting model can take time ...
Fitting model completed.

Preparing Predictions

Metrics : After Optuna Tuning
MAE: 0.026901392674457677
RMSE: 0.16401644025663303

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    107165
           1       0.99      0.95      0.97    107099

    accuracy                           0.97    214264
   macro avg       0.97      0.97      0.97    214264
weighted avg       0.97      0.97      0.97    214264


Confusion Matrix:

True Negative : CHGOFF (0) was predicted 106332 times correctly   (95.57 %)
False Negative : CHGOFF (0) was predicted 4931 times incorrectly     (4.43 %)
True Positive : P I F (1) was predicted 102168 times correctly     (99.19 %)
False Positive : P I F (1) was predicted 833 times 

In [ ]:
if alert_flag == 1:
    if kaggle_flag == 0:   # not Kaggle
        engine.say("Model Test with Optuna completed.")
        engine.runAndWait()
    else:
        display(Audio(url=audio_path, autoplay=True))

<a id="optuna_comparison"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>1.5 Optuna Tuning Comparison</h2>
</div>

<div class="alert alert-block alert-info">
    <b>Compare metrics before and after Optuna tuning.</b><br><br>
    Comparison is made between modelv3 results in <a style="color:DarkSlateGrey" href="https://www.kaggle.com/code/josephramon/sba-xgboost-model">Part 1 Notebook</a> and modelv4 results here.
  </div>

In [ ]:
def CompareResults():
    model3_results = joblib.load(f"{filepath}model3_results.dict")
    model4_results = joblib.load(f"{savepath}model4_results.dict")

    m3_clf_report = model3_results['eval_results']['ClassificationReport']

    m3_0_f1_score = round(m3_clf_report['0']['f1-score'] * 100, 2)
    m3_1_f1_score = round(m3_clf_report['1']['f1-score'] * 100, 2)
    m3_accuracy   = round(m3_clf_report['accuracy'] * 100, 2)


    m4_clf_report = model4_results['eval_results']['ClassificationReport']

    m4_0_f1_score = round(m4_clf_report['0']['f1-score'] * 100, 2)
    m4_1_f1_score = round(m4_clf_report['1']['f1-score'] * 100, 2)
    m4_accuracy   = round(m4_clf_report['accuracy'] * 100, 2)


    data = {'Model v3 : No Optuna':[m3_0_f1_score, m3_1_f1_score, m3_accuracy],
            'Model v4 : With Optuna':[m4_0_f1_score, m4_1_f1_score, m4_accuracy]}
 
    # Creates pandas DataFrame.
    df = pd.DataFrame(data, index =['0 : f1_score',
                                    '1 : f1_score',
                                    'Accuracy'])
    print(f'{color.bdgreen}Accuracy Improvement Using Optuna Suggested Parameters:{color.end}')
    print(f'{color.bold}Improved by {color.bdblue}{round(m4_accuracy - m3_accuracy,2)}\
        {color.end}')
    return df

CompareResults()

<div class="alert alert-block alert-info">
    <b>Run Modelv4 On Unseen Test Dataset</b>
  </div>

In [21]:
def Modelv4WithTestData():
    model4_results = joblib.load(f"{savepath}model4_results.dict")
    X_test = model4_results['X_test']
    y_test = model4_results['y_test']
    modelv4 = model4_results['xg_model']
    
    # Get predictions
    predictions = modelv4.predict(X_test)
    model_eval(y_test, predictions);
  
Modelv4WithTestData()

MAE: 0.026663368554680208
RMSE: 0.16328921750893477

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    107332
           1       0.99      0.95      0.97    106932

    accuracy                           0.97    214264
   macro avg       0.97      0.97      0.97    214264
weighted avg       0.97      0.97      0.97    214264


Confusion Matrix:

True Negative : CHGOFF (0) was predicted 106565 times correctly   (95.56 %)
False Negative : CHGOFF (0) was predicted 4946 times incorrectly     (4.44 %)
True Positive : P I F (1) was predicted 101986 times correctly     (99.25 %)
False Positive : P I F (1) was predicted 767 times incorrectly     (0.75 %)

Accuracy for model: 97.33
f1-score: 
   CHGOFF (0) : 97.39
   P I F (1)  : 97.28
RMSE: 0.16328921750893477


In [ ]:
del best_trial, study_results
gc.collect()

<div class="alert alert-block alert-info">
    <b>Observation:</b><br><br>
    <b>The Accuracy and F1 scores after Optuna tuning are improved compared to before tuning;</b> but it all depends on what hyperparameters/values are given.  A few trial sessions may be needed.<br><br>
    We have a different score in our <a style="color:DarkSlateGrey" href="https://www.kaggle.com/code/josephramon/sba-xgboost-model">Pipeline</a> as we used an Optuna hyperparameter set that was obtained from another Optuna run.
  </div>

<a id="cross_validation"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 20px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>Cross Validation</h2><br>
Measure our model's quality, in RMSE.  Ideally for small datasets, but included here for reference.
</div>

In [ ]:
%%time

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

def CrossVal():
    print(f'{color.bold}Please wait, this will take some time{color.end}')
    print()
    X = pd.read_feather(final_ds)
    y = X.pop('MIS_Status')

    cvo = process_model(X, y)   # create object from XGBoost class
    cvo.osample()               # oversample
    cvo.split_data(0.7)

    cvo.X, cvo.y = None, None
    del X, y
    gc.collect()
    sleep(3)
    
    print()
    
    # get hyperparams from previous Optuna study's best trial
    study_results = joblib.load(f"{savepath}xgb_optuna_study.pkl")
    hyperparams = study_results.best_trial.params
    pprint(hyperparams)
    print()

    xgb_model = XGBClassifier(**hyperparams, use_label_encoder = False)

    # If we pass a pipeline instead of a model to cross_val_score, fit_params won't be 
    # recognized
    fit_params={'verbose': False,
                'eval_set': [(cvo.X_valid, cvo.y_valid)]
               }

    # Multiply by -1 since sklearn calculates *negative* RMSE
    print()
    scores = -1 * cross_val_score(xgb_model, cvo.X_train, cvo.y_train,
                                  cv=5,
                                  scoring='neg_root_mean_squared_error',
                                  fit_params = fit_params,
                                  verbose=15)
    sleep(5)   # allow verbosity to complete
    print()
    print(f"{color.bdblue}Scores: {scores}{color.end}")
    print()
    print(f"{color.bdgreen}Root Mean Squared Error (Mean): {scores.mean()}{color.end}")
    print()
    
CrossVal()

<a id="part2"></a>
<div style="font-family: Trebuchet MS;background-color:DarkRed;color:AliceBlue;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
<h1 id="part2" style='color:GhostWhite;'>Part 2. Miscellaneous</h1>
</div>

<a id="early_stopping_rounds"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.1 Early Stopping Rounds</h2>
</div>

<div class="alert alert-block alert-info">
    Below is a reference on using plots to get an insight on the value to use for XGBoost's early_ stopping_rounds during fitting.</div>

In [ ]:
%%time

from matplotlib import pyplot

def PlotEarlyStoppingRounds():
    X = pd.read_feather(final_ds)
    y = X.pop('MIS_Status')
    
    esr = process_model(X, y)
    esr.osample()     # oversample
    esr.split_data(0.7)
    
    esr.X, esr.y = None, None
    del X, y
    gc.collect()
    sleep(3)   
    
    print()
    print(f"{color.bold}Please wait, Fitting model can take time ...{color.end}")
    print()
    
    # get hyperparams from previous Optuna study's best trial
    # early_stopping_rounds not included as we are trying to confirm if our value was valid
    study_results = joblib.load(f"{savepath}xgb_optuna_study.pkl")
    hyperparams = study_results.best_trial.params
    hyperparams.pop('early_stopping_rounds')
    hyperparams['eval_metric'] = ['error','logloss']
    pprint(hyperparams)
    print()

    xg_model = XGBClassifier(**hyperparams,use_label_encoder =False)
       
    eval_setparam = [(esr.X_train, esr.y_train),
                     (esr.X_valid, esr.y_valid)]
       
    # fit the model
    xg_model.fit(esr.X_train, esr.y_train, 
                eval_set = eval_setparam,
                verbose=False)
 
    print("Fitting model completed.")
    print()
    print('Preparing Predictions')
    
    # Get predictions
    predictions = xg_model.predict(esr.X_valid)
    
    print()
    print(f'{color.underline}Metrics:{color.end}')

    eval_results = model_eval(esr.y_valid, predictions)

    # retrieve performance metrics
    results = xg_model.evals_result()
    epochs = len(results['validation_0']['error'])
    x_axis = range(0, epochs)

    # what we will be looking for are the bottom areas of the plots
    
    # plot log loss
    fig, ax = pyplot.subplots()
    ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
    ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
    ax.legend()
    pyplot.xlabel('Epoch')
    pyplot.ylabel('Log Loss')
    pyplot.title('XGBoost Log Loss')
    pyplot.show()

    # plot classification error
    fig, ax = pyplot.subplots()
    ax.plot(x_axis, results['validation_0']['error'], label='Train')
    ax.plot(x_axis, results['validation_1']['error'], label='Test')
    ax.legend()
    pyplot.xlabel('Epoch')
    pyplot.ylabel('Classification Error')
    pyplot.title('XGBoost Classification Error')
    pyplot.show()
    
PlotEarlyStoppingRounds()

<div class="alert alert-block alert-info">
    From both plots, we can see that 10% of n_estimator is a good candidate as the early_stopping_rounds parameter.
<br><br>
    <b>Test Early Stopping Rounds</b>
    <br>
    Now we test the early_stopping_rounds parameter.  We will test it on imbalanced data, which is our data that is <b>not oversampled</b>, which is skewed heavily toward MIS_Status = 'P I F' as opposed to MIS_Status = 'CHGOFF'.  Furthermore, we use an <b>eval_metric = 'auc'</b>, which is ideal for imbalanced data.
</div>

In [23]:
%%time

def RunModelv5():
    X = pd.read_feather(final_ds)
    y = X.pop('MIS_Status')

    model5 = process_model(X, y)
    # model5.osample()     # we comment out the oversampling algorithm
    model5.split_data(0.7)
    
    model5.X, model5.y = None, None
    del X, y
    gc.collect()
    sleep(3)
    
    # we use the recently saved optuna study to get the hyperparams
    study = joblib.load(f"{savepath}xgb_optuna_study.pkl")
    hyperparams = study.best_trial.params
    hyperparams['eval_metric'] = 'auc'      # we change eval_metric to 'auc'
                   
    model_xg = XGBClassifier(**hyperparams, use_label_encoder =False)
 
    print(f'{color.bold}Fit() Iterations:{color.end}')
    eval_set = [(model5.X_valid, model5.y_valid)]
    model_xg.fit(model5.X_train, model5.y_train,
              eval_set=eval_set, verbose=True)
 
    # Get predictions
    predictions = model_xg.predict(model5.X_valid)

    print()
    eval_results = model_eval(model5.y_valid, predictions)
    
RunModelv5()

MIS_Status Count ->  1 : 714212, 0 : 154451
Fit() Iterations:
[0]	validation_0-auc:0.96545
[1]	validation_0-auc:0.96773
[2]	validation_0-auc:0.96848
[3]	validation_0-auc:0.96479
[4]	validation_0-auc:0.96663
[5]	validation_0-auc:0.96780
[6]	validation_0-auc:0.96846
[7]	validation_0-auc:0.96950
[8]	validation_0-auc:0.96977
[9]	validation_0-auc:0.97004
[10]	validation_0-auc:0.97019
[11]	validation_0-auc:0.97050
[12]	validation_0-auc:0.97063
[13]	validation_0-auc:0.97075
[14]	validation_0-auc:0.97019
[15]	validation_0-auc:0.97035
[16]	validation_0-auc:0.96974
[17]	validation_0-auc:0.97001
[18]	validation_0-auc:0.97023
[19]	validation_0-auc:0.97051
[20]	validation_0-auc:0.97066
[21]	validation_0-auc:0.97030
[22]	validation_0-auc:0.97047
[23]	validation_0-auc:0.97023
[24]	validation_0-auc:0.97043
[25]	validation_0-auc:0.97065
[26]	validation_0-auc:0.97082
[27]	validation_0-auc:0.97095
[28]	validation_0-auc:0.97108
[29]	validation_0-auc:0.97121
[30]	validation_0-auc:0.97133
[31]	validation_0-

[266]	validation_0-auc:0.97481
[267]	validation_0-auc:0.97481
[268]	validation_0-auc:0.97481
[269]	validation_0-auc:0.97481
[270]	validation_0-auc:0.97481
[271]	validation_0-auc:0.97480
[272]	validation_0-auc:0.97480
[273]	validation_0-auc:0.97481
[274]	validation_0-auc:0.97481
[275]	validation_0-auc:0.97481
[276]	validation_0-auc:0.97480
[277]	validation_0-auc:0.97480
[278]	validation_0-auc:0.97481
[279]	validation_0-auc:0.97480
[280]	validation_0-auc:0.97480
[281]	validation_0-auc:0.97480
[282]	validation_0-auc:0.97480
[283]	validation_0-auc:0.97480
[284]	validation_0-auc:0.97480
[285]	validation_0-auc:0.97480
[286]	validation_0-auc:0.97480
[287]	validation_0-auc:0.97480
[288]	validation_0-auc:0.97481
[289]	validation_0-auc:0.97481
[290]	validation_0-auc:0.97480
[291]	validation_0-auc:0.97481
[292]	validation_0-auc:0.97481
[293]	validation_0-auc:0.97480
[294]	validation_0-auc:0.97480
[295]	validation_0-auc:0.97480
[296]	validation_0-auc:0.97481
[297]	validation_0-auc:0.97481
[298]	va

<div class="alert alert-block alert-info">
    <b>Result:</b><br>
    As we can see from above, from the verbose results of the fit() method, <b>the training stopped at a certain iteration before it reached 1000</b>, since <b>the auc score was starting to decrease</b>.  In conclusion, the early_stopping_rounds parameter works.  Although it was set at 100, at 100th iteration, the <b>auc score was still increasing</b>, so it didn't stop yet.
    <br><br>
    If we oversampled the data, the fit() method will complete the entire n_estimators = 1000, as the score is still getting better even at 1000.  In conclusion, oversampling helped get better results without reaching the overfitting threshold yet at the selected n_estimator value.  It's not just blind oversampling of course, it was a technique to solve the imbalance of data.
</div>

<a id="random_forest_classifier"></a>
<div style="font-family: Trebuchet MS;background-color:DarkCyan;color:Azure;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <h2 style='color:GhostWhite;'>2.2 Random Forest Classifier</h2>
</div>

<div class="alert alert-block alert-info">
    This is just a reference on using a Random Forest Classifier.<br>
    <b>cuml.ensemble RandomForestClassifier</b> is used if GPU is active; otherwise, Scikit-Learn's RandomForestClassifier.
  </div>

In [32]:
%%time

# run before tuning
def RunModelrf():
    X = pd.read_feather(final_ds)
    y = X.pop('MIS_Status')

    modelrf = other_models(X, y)
    modelrf.osample()
    modelrf.split_data(0.7)
    
    modelrf.X, modelrf.y = None, None
    del X, y
    gc.collect()
    sleep(3)
    
    modelrf.prep_run_model("Metrics : Random Forest Classifier", modelname='rfc')

RunModelrf()

X size :  868663
y size :  868663
Before Oversampling -> 1 : 714212, 0 : 154451
After Oversampling  -> 1 : 714212, 0 : 714212

Please wait, Fitting model can take time ...
Fitting model completed.

Preparing Predictions
Metrics : Random Forest Classifier
MAE: 15.178788783930106
RMSE: 0.3341785696780467

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.90      0.89    107165
           1       0.89      0.88      0.89    107099

    accuracy                           0.89    214264
   macro avg       0.89      0.89      0.89    214264
weighted avg       0.89      0.89      0.89    214264


Confusion Matrix:

True Negative : CHGOFF (0) was predicted 95947 times correctly   (88.3 %)
False Negative : CHGOFF (0) was predicted 12710 times incorrectly     (11.7 %)
True Positive : P I F (1) was predicted 94389 times correctly     (89.38 %)
False Positive : P I F (1) was predicted 11218 times incorrectly     (10.62 %)

Accuracy for mod

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Optuna Tuning for Random Forest</b>
</div>

<div class="alert alert-block alert-info">  
    This is just a simple sample implementation, for reference.</div>

In [ ]:
%%time

def RandomForestOptunaTuning():
    X = pd.read_feather(final_ds)
    y = X.pop('MIS_Status')

    rfo = optuna_tuning(X, y)
    rfo.osample()  # oversample
    rfo.split_data(0.7)
    
    rfo.X, rfo.y = None, None
    del X, y
    gc.collect()
    sleep(3)
    
    print(f'{color.bold}Please wait, this will take time{color.end}')
    
    nn_trials = 3
    
    if os.path.exists(f'{savepath}rfc_optuna_study_log.txt'):
        os.remove(f'{savepath}rfc_optuna_study_log.txt') 
    
    # STUDY
    
    # Turn off optuna log notes, to use own logging notes
    optuna.logging.set_verbosity(optuna.logging.WARN)
    
    # Turn on optuna log notes
    #optuna.logging.set_verbosity(optuna.logging.INFO)

    print(f"{color.bold}Please wait, finding best trial ...{color.end}")
            
    gt = dt.datetime.now()
    
    study = optuna.create_study(direction="maximize")
    
    try:
        # callbacks [self.save_study] is to save study in case memory fails
        study.optimize(lambda trial: rfo.objective_rf(trial, gt), 
                        n_trials = nn_trials,
                        callbacks = [rfo.logging_callback, rfo.save_study],
                        gc_after_trial = True,
                        catch = (RuntimeWarning,ArithmeticError,))
    except MemoryError as e:
        print(f'{color.bdblue}{e} : Memory was getting low, Trial ended early{color.end}')
  
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    print()
    print(f'Number of finished trials: {len(study.trials)}')
    print(f'Number of pruned trials: {len(pruned_trials)}')
    print(f'Number of completed trials: {color.bdgreen}{len(complete_trials)}{color.end}')
    print(f'{color.bdblue}Best trial: {study.best_trial.params}{color.end}')
    
    joblib.dump(study, f"{savepath}rfc_optuna_study.pkl")   # save study
    # jl = joblib.load(f"{savepath}rfc_optuna_study.pkl")   # load study

    print()
    return study

study_results = RandomForestOptunaTuning()

Trial 0 done with best value: 0.8762274577157152 and parameters: {'max_features': 0.660357966635801, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_samples': 0.6606472757270871, 'max_depth': 5, 'n_estimators': 100}. 

Current Ram Used: 71.4 %
Total Elapsed Time from Training Start: Runtime : 0:03:46.949617
Running Trial 2


In [ ]:
pprint(study_results.best_params) # Get best parameters for the objective function.
print()
pprint(study_results.best_value)  # Get best objective value.
print()
pprint(study_results.best_trial)  # Get best trial's information.

In [ ]:
best_trial = study_results.best_trial.params
best_trial

In [ ]:
# Trial results dataframe sorted from best value ascending
ViewResultsAsDf()

In [ ]:
#Visualize parameter importance
optuna.visualization.plot_param_importances(study_results)

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>Random Forest Score With Optuna Hyperparameters</b>
</div>

In [ ]:
%%time

def RunModelrf2():
    X = pd.read_feather(final_ds)
    y = X.pop('MIS_Status')

    modelrf = other_models(X, y)
    modelrf.osample()
    modelrf.split_data(0.7)
    
    modelrf.X, modelrf.y = None, None
    del X, y
    gc.collect()
    sleep(3)
    
    modelrf.prep_run_model("Metrics : Random Forest Classifier",
                            modelname='rfc', hparams = best_trial)

RunModelrf2()

<div class="alert alert-block alert-info">
    Once again, Optuna helped us get parameters that improved the score.</div>

In [ ]:
del study_results, best_trial
gc.collect()

<div style="font-family: Trebuchet MS;background-color:LightSteelBlue;color:Black;text-align: left;padding-top: 5px;padding-bottom: 15px;padding-left: 20px;padding-right: 10px;border-radius: 15px 50px;letter-spacing: 2px;">
    <b>RandomizedSearchCV</b>
</div>

<div class="alert alert-block alert-info">
    Below is a reference on using a <b>RandomizedSearchCV</b> first for Random Forest hyperparameter tuning.<br><br>
  Once done, one would have randomly narrowed down some parameters which we can base our inputs for a full <b>GridSearchCV</b> (not shown here).
    <br><br>
    Both approaches take an <b>extremely long time to run</b> using our SBA dataset, and the line to run the task is commented out.  Uncomment if you want to try.  Otherwise, <b>Optuna</b> is a much faster method.</div>

In [ ]:
def ViewDefaultRFCParams():
    rf = RandomForestClassifier(random_state = 48)
    # Look at parameters used by our current forest
    print('Default parameters in use:\n')
    pprint(rf.get_params())

ViewDefaultRFCParams()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

def SuggestRFCParams():
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 500, stop = 2000, num = 3)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(6, 15, num = 4)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    pprint(random_grid)
    return random_grid

random_grid = SuggestRFCParams()

In [ ]:
%%time

def RandomSearchCV(random_grid):
    X = pd.read_feather(final_ds)
    y = X.pop('MIS_Status')

    modelrf = other_models(X, y)
    modelrf.osample()
    modelrf.split_data(0.7)
    
    X_train, y_train = modelrf.X_train, modelrf.y_train
    
    del X, y, modelrf
    gc.collect()
    sleep(3)
    
    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestClassifier()

    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                                   n_iter = 5, cv = 3, verbose=10, random_state=48)

    # Fit the random search model
    rf_random.fit(X_train, y_train)
    
    return rf_random.best_params_

#rf_best_params = RandomSearchCV(random_grid)
#rf_best_params

In [ ]:
del random_grid #,rf_best_params 
gc.collect()